# Setup

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import csv

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from ipywidgets import interact

from flask import Flask, jsonify


In [ ]:
# create and save database
#execfile("Setup and Organization/Database Setup.py")

In [2]:
# create engine to newly created database.sqlite
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()


# Explore Database

In [3]:
# explore database
inspector = inspect(engine)
inspector.get_table_names()

['activity',
 'bmi',
 'gender',
 'health',
 'occupation',
 'person',
 'sleep',
 'sleep_disorder']

In [4]:
inspector.get_columns('activity')

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'person_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'activity_level',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'daily_steps',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

# Reflect and query data

In [6]:
# reflect an existing database into a new model
base= automap_base()
# reflect the tables
base.prepare(autoload_with=engine)

In [8]:
base.classes.keys()

[]

In [ ]:
#activity = Base.classes.activity

In [10]:
base.metadata.tables

FacadeDict({'activity': Table('activity', MetaData(), Column('index', BIGINT(), table=<activity>), Column('id', BIGINT(), table=<activity>), Column('person_id', BIGINT(), table=<activity>), Column('activity_level', BIGINT(), table=<activity>), Column('daily_steps', BIGINT(), table=<activity>), schema=None), 'bmi': Table('bmi', MetaData(), Column('index', BIGINT(), table=<bmi>), Column('id', BIGINT(), table=<bmi>), Column('bmi_name', TEXT(), table=<bmi>), schema=None), 'gender': Table('gender', MetaData(), Column('index', BIGINT(), table=<gender>), Column('id', BIGINT(), table=<gender>), Column('gender_name', TEXT(), table=<gender>), schema=None), 'health': Table('health', MetaData(), Column('index', BIGINT(), table=<health>), Column('id', BIGINT(), table=<health>), Column('person_id', BIGINT(), table=<health>), Column('blood_pressure', TEXT(), table=<health>), Column('stress_level', BIGINT(), table=<health>), Column('heart_rate', BIGINT(), table=<health>), Column('systolic', BIGINT(), 

In [12]:
for mappedclass in base.classes:
    print(mappedclass)



In [14]:
for mdtable in base.metadata.tables:
    print(mdtable)

activity
bmi
gender
health
occupation
person
sleep
sleep_disorder


In [17]:
Person= pd.read_sql("SELECT * FROM person", conn)
Sleep= pd.read_sql("SELECT * FROM sleep", conn)
Health= pd.read_sql("SELECT * FROM health", conn)

In [35]:
health_metrics = pd.read_sql(f'select * from person as p \
                             JOIN health as h on p.id = h.person_id \
                             JOIN bmi on h.bmi_id = bmi_id \
                             JOIN gender as g on p.gender_id = g.id', conn)
health_metrics

,index,id,gender_id,age,occupation_id,index,id,person_id,blood_pressure,stress_level,heart_rate,systolic,diastolic,bmi_id,index,id,bmi_name,index,id,gender_name
0,0,1,0,27,0,0,0,1,126/83,6,77,126,83,0,0,0,Overweight,0,0,Male
1,0,1,0,27,0,0,0,1,126/83,6,77,126,83,0,1,1,Normal,0,0,Male
2,0,1,0,27,0,0,0,1,126/83,6,77,126,83,0,2,2,Obese,0,0,Male
3,369,69,1,33,7,1,1,69,128/85,6,76,128,85,0,0,0,Overweight,1,1,Female
4,369,69,1,33,7,1,1,69,128/85,6,76,128,85,0,1,1,Normal,1,1,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,71,277,0,49,1,372,372,277,139/91,3,86,139,91,2,1,1,Normal,0,0,Male
1118,71,277,0,49,1,372,372,277,139/91,3,86,139,91,2,2,2,Obese,0,0,Male
1119,72,278,0,49,1,373,373,278,139/91,3,86,139,91,2,0,0,Overweight,0,0,Male
1120,72,278,0,49,1,373,373,278,139/91,3,86,139,91,2,1,1,Normal,0,0,Male


In [37]:
health_metrics = health_metrics[['person_id','heart_rate','systolic','diastolic']]
health_metrics

,person_id,heart_rate,systolic,diastolic
0,1,77,126,83
1,1,77,126,83
2,1,77,126,83
3,69,76,128,85
4,69,76,128,85
...,...,...,...,...
1117,277,86,139,91
1118,277,86,139,91
1119,278,86,139,91
1120,278,86,139,91
